### Set up

#### 1. Set  up  accounts and role

In [ ]:
!pip install sagemaker==1.39.0

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [20]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201909140804"
instance_type = "ml.p3.8xlarge" 

In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [4]:
bucket = "aegovan-data"

In [5]:
trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_less_negative.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_unique_pub_v6_less_negative.json".format(bucket)
valfile="s3://{}/processed_dataset/val_unique_pub_v6_less_negative.json".format(bucket)
embeddingfile="s3://{}/embeddings/wikipedia-pubmed-and-PMC-w2v.bin.txt".format(bucket)
s3_output_path= "s3://{}/results/".format(bucket)

### Start training

In [6]:
inputs = {
    "train" : trainfile,
    "val" :valfile,
    "embedding" : embeddingfile
}

In [24]:
hyperparameters = {
    "dataset":"PPIDataset",
    "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
    "embeddim":200,
    "batchsize": "32",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "lstmhiddensize": 50,
     "fclayersize": 50,
    "numlayers":3,
    "poolingkernelsize":4
   
}

In [25]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [26]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
              'commit': 'c050a2818efd6dc6d2adb3dac07c102b94c92eab'}

In [27]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     entry_point='main_train.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                   # git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    base_job_name ="ppi-extractor")

In [ ]:
estimator.fit(inputs)

2019-09-15 11:10:11 Starting - Starting the training job...
2019-09-15 11:10:13 Starting - Launching requested ML instances...
2019-09-15 11:11:07 Starting - Preparing the instances for training......
2019-09-15 11:12:11 Downloading - Downloading input data..................
2019-09-15 11:15:49 Training - Downloading the training image.........
2019-09-15 11:17:48 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-09-15 11:17:49,492 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-09-15 11:17:49,535 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-09-15 11:17:55,751 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-09-15 11:17:56,033 sagemaker-containers INFO     Module main_train does not provide a setup.py. 
Generating setup.py
2019-09-1

2019-09-15 11:25:29,604 - algorithms.PretrainedEmbedderLoader - INFO - The number of words intialised without embbeder is 118
2019-09-15 11:26:50,545 - algorithms.transform_protein_mask - INFO - Running TransformProteinMask 
2019-09-15 11:26:54,801 - algorithms.transform_protein_mask - INFO - Completed TransformProteinMask 
2019-09-15 11:26:54,801 - algorithms.transform_protein_mask - INFO - Running TransformProteinMask 
2019-09-15 11:26:54,810 - algorithms.transform_protein_mask - INFO - Completed TransformProteinMask 
2019-09-15 11:26:54,810 - algorithms.transform_sentence_tokeniser - INFO - Running sentence tokenisor 
2019-09-15 11:28:53,309 - algorithms.transform_sentence_tokeniser - INFO - Completed  sentence tokenisor 
2019-09-15 11:28:53,315 - algorithms.transform_text_index - INFO - Transforming TransformTextToIndex
2019-09-15 11:28:53,843 - algorithms.transform_text_index - INFO - Completed TransformTextToIndex
2019-09-15 11:28:53,845 - algorithms.transform_protein_mask - INFO

2019-09-15 11:43:18,802 - algorithms.Train - INFO - Train set result details:
2019-09-15 11:43:18,806 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_58586081-5499-469f-915c-6407f16dac0d_20190915_114318.csv: 
[[2742   58]
 [ 754   85]]
2019-09-15 11:43:18,808 - algorithms.Train - INFO - Train set result details: 0.17311608961303462
2019-09-15 11:43:18,808 - algorithms.Train - INFO - Validation set result details:
2019-09-15 11:43:37,391 - algorithms.result_writer - INFO - Confusion matrix, full output in /opt/ml/output/data/predictedvsactual_0493ec9e-d81a-4ae0-a570-b5e0f66344d9_20190915_114337.csv: 
[[347  27]
 [ 78  27]]
2019-09-15 11:43:37,393 - algorithms.Train - INFO - Validation set result details: 0.33962264150943394 
2019-09-15 11:43:37,393 - algorithms.Train - INFO - Run    856     4       570     5/114         4% 56.766930 7.547141       0.1731       0.3396
###score: train_loss### 56.766929626464844
###score: val_loss#

### HPO

In [15]:
objective_metric_name ="ValidationFScore"

In [16]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
hyperparameter_ranges = {'lstmhiddensize': IntegerParameter(40,200), #ContinuousParameter(0.01, 0.2),
                          "fclayersize": IntegerParameter(10,50),
                            "numlayers":IntegerParameter(1,10),
                        "poolingkernelsize":IntegerParameter(2,5),
                         'num_epoch': IntegerParameter(10, 50)}

In [17]:
hyperparameters={ "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
                        "embeddim":"200",
                        "dataset":"PPIDataset"}

In [21]:

estimator = PyTorch(
     entry_point='main_train.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                   # git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    base_job_name ="hpo-ppi-extractor")

In [22]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=9,
                            max_parallel_jobs=3,
                            base_tuning_job_name="hpo-ppi-extractor")
tuner.fit(inputs)